# import

In [213]:
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.matlib
from pathlib import Path
from scipy import constants
from scipy.linalg import block_diag

# 芯片参数设计

## 基本参数
电容单位为fF

In [188]:
# C_q表示比特自电容，外层index表示周围有几个coupler1、2、3、4,内层表示每个岛的子电容，最后一项表示岛间电容
C_q = np.array([[80]*2+[9], [80]*2+[9], [80]*2+[9], [80]*2+[9]])
# C_c表示coupler自电容，外层index表示coupler类型，内层表示每个岛的子电容，最后一项表示岛间电容
C_c = np.array([[90]*2+[8], [90]*2+[8]])
# C_qc表示qc耦合电容，最外层表示coupler类型，中层表示比特的每个岛，内层表示对coupler的每个岛的电容
C_qc = np.array([[[5, 5], [5, 5]], [[5, 5], [5, 5]]])
# C_qc表示qq耦合电容，外层表示第一个比特的岛，内层表示第二个比特的岛
C_qq = np.array([[0.5, 0.5], [0.5, 0.5]])
# n_q为每行的比特个数
shape_q = [12, 6]

In [82]:
C_q

array([[80, 80,  5],
       [80, 80,  5],
       [80, 80,  5],
       [80, 80,  5]])

In [80]:
[80, 80].extend([5])

## 计算电容矩阵
节点顺序为比特、coupler I型、coupler II型。。。

In [138]:
types_c = len(C_qc)
n_qi = len(C_qc[0])  # 比特的岛的个数，默认所有比特全同
n_ci_list = [len(C_qc[i][0])
             for i in range(len(C_qc))]  # 每种类型的coupler的岛的个数

N_q = n_q**2 * n_qi
N_c = 2 * n_q * (n_q - 1) * sum(n_ci_list)
# 每类coupler的第一个coupler的节点编号
c_start_index_list = [
    N_q + 2 * n_q * (n_q - 1) * sum(n_ci_list[:j])
    for j in range(len(n_ci_list))
]

C_arr_ori = np.matlib.zeros((N_q + N_c, N_q + N_c))
i = 0
row = i % n_q
column = i // n_q

q_i_index_list = range(n_qi * i, n_qi * (i + 1))

q_i_index_list1 = slice(n_qi * i, n_qi * (i + 1))

if row < n_q - 1:
    # 当前比特的下方比特的不同岛的节点编号
    q_j_index_list = range(n_qi * (i + n_q),
                                n_qi * (i + 1 + n_q))
    
    q_j_index_list1 = slice(n_qi * (i + n_q),
                                n_qi * (i + 1 + n_q))
    C_arr_ori[q_i_index_list, :][:, q_j_index_list] = -C_qq
    print(C_arr_ori[q_i_index_list, :][:, q_j_index_list])
    # 当前比特的下方coupler的不同类型不同岛的节点编号
    c_j_index = row * (2 * n_q -
                       1) + n_q - 1 + column  # 该coupler在本类coupler中的编号

[[0. 0.]
 [0. 0.]]


In [141]:
C_arr_ori[q_i_index_list, q_j_index_list] = -C_qq[0]
C_arr_ori[q_i_index_list, q_j_index_list]

matrix([[-0.5, -0.5]])

In [140]:
q_i_index_list

range(0, 2)

In [128]:
C_arr_ori[q_i_index_list1, q_j_index_list1] = -C_qq
C_arr_ori[q_i_index_list, :][:, q_j_index_list]

matrix([[-0.5, -0.5],
        [-0.5, -0.5]])

In [125]:
C_arr_ori[q_i_index_list, :][:, q_j_index_list]

matrix([[0., 0.],
        [0., 0.]])

In [64]:
slice(n_qi * (i + n_q), n_qi * (i + 1 + n_q))

TypeError: 'slice' object is not iterable

In [150]:
a = np.zeros((3,3))
a[0:2,0:1] = np.ones((2,1))
a

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.]])

In [152]:
a

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.]])

In [133]:
a = np.zeros((3,3))
a[[0,1],:][:,[0,1]] = np.ones((2,2))
a

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [411]:
def C_matrix_generate(shape_q, C_q, C_c, C_qc):
    row_max = shape_q[0]
    column_max = shape_q[1]
    n_qi = len(C_qc[0])  # 比特的岛的个数，默认所有比特全同
    n_ci_list = [len(C_qc[i][0]) for i in range(len(C_qc))]  # 每种类型的coupler的岛的个数

    N_q = np.prod(shape_q) * n_qi
    N_c = (2 * row_max * column_max - row_max - column_max) * sum(n_ci_list)
    # 每类coupler的第一个coupler的节点编号
    c_start_index_list = [
        N_q +
        (2 * row_max * column_max - row_max - column_max) * sum(n_ci_list[:j])
        for j in range(len(n_ci_list))
    ]

    C_arr_ori = np.matlib.zeros((N_q + N_c, N_q + N_c))
    C_list = range(N_q + N_c)
    for i in range(row_max * column_max):
        row = i // column_max
        column = i % column_max

        # 当前比特的不同岛的节点编号
        q_i_index_slice = slice(n_qi * i, n_qi * (i + 1))
        q_i_index_list = C_list[q_i_index_slice]

        if row < row_max - 1:
            # 当前比特的下方比特的不同岛的节点编号
            q_j_index_slice = slice(n_qi * (i + column_max),
                                    n_qi * (i + 1 + column_max))
            q_j_index_list = C_list[q_j_index_slice]

            C_arr_ori[q_i_index_slice, q_j_index_slice] = -C_qq

            # 当前比特的下方coupler的不同类型不同岛的节点编号
            c_j_index = row * (
                2 * column_max -
                1) + column_max - 1 + column  # 该coupler在本类coupler中的编号

            for j, c_start_index in enumerate(c_start_index_list):

                c_j_index_slice = slice(
                    c_start_index + n_ci_list[j] * c_j_index,
                    c_start_index + n_ci_list[j] * (c_j_index + 1))
                c_j_index_list = C_list[c_j_index_slice]

                # 当前比特对下方j类coupler
                C_arr_ori[q_i_index_slice, c_j_index_slice] = -C_qc[j]
                # 下方比特对下方j类coupler
                C_arr_ori[q_j_index_slice, c_j_index_slice] = -C_qc[j]
                # 设置下方coupler自电容
                if n_ci_list[j] == 1:
                    C_arr_ori[c_j_index_list, c_j_index_list] = C_c[j]  # 设置岛对地电容
                else:
                    C_arr_ori[c_j_index_list,
                              c_j_index_list] = C_c[j][:-1]  # 设置岛对地电容
                    C_arr_ori[c_j_index_list[0], c_j_index_list[-1]] = -C_c[j][-1]

        if column < column_max - 1:
            # 当前比特的右方比特的不同岛的节点编号
            q_j_index_slice = slice(n_qi * (i + 1), n_qi * (i + 2))
            q_j_index_list = C_list[q_j_index_slice]

            C_arr_ori[q_i_index_slice, q_j_index_slice] = -C_qq
            # 当前比特的右方coupler的不同类型不同岛的节点编号
            c_j_index = row * (2 * column_max -
                               1) + column  # 该coupler在本类coupler中的编号

            for j, c_start_index in enumerate(c_start_index_list):

                c_j_index_slice = slice(
                    c_start_index + n_ci_list[j] * c_j_index,
                    c_start_index + n_ci_list[j] * (c_j_index + 1))

                c_j_index_list = C_list[c_j_index_slice]
                # 当前比特对右方j类coupler
                C_arr_ori[q_i_index_slice, c_j_index_slice] = -C_qc[j]
                # 右方比特对右方j类coupler
                C_arr_ori[q_j_index_slice, c_j_index_slice] = -C_qc[j]
                # 设置右方coupler自电容
                if n_ci_list[j] == 1:
                    C_arr_ori[c_j_index_list, c_j_index_list] = C_c[j]  # 设置岛对地电容
                else:
                    C_arr_ori[c_j_index_list,
                              c_j_index_list] = C_c[j][:-1]  # 设置岛对地电容
                    C_arr_ori[c_j_index_list[0], c_j_index_list[-1]] = -C_c[j][-1]

        # 设置比特自电容
        nq_around = 0  # 比特周围的比特数
        if row < row_max - 1:
            nq_around += 1
        if row > 0:
            nq_around += 1
        if column < column_max - 1:
            nq_around += 1
        if column > 0:
            nq_around += 1

        if n_qi == 1:
            C_arr_ori[q_i_index_list, q_i_index_list] = C_q[nq_around - 1]
        else:
            C_arr_ori[q_i_index_list, q_i_index_list] = C_q[nq_around - 1][:-1]
            C_arr_ori[q_i_index_list[0],
                      q_i_index_list[-1]] = -C_q[nq_around - 1][-1]

    # 设置下三角区域元素
    C_down = C_arr_ori.T
    C_arr_ori = C_arr_ori + C_down
    C_arr_ori[C_list, C_list] = C_arr_ori[C_list, C_list] / 2

    # 把非对角元加到对角元上
    C_arr_ori[C_list,
              C_list] = 2 * C_arr_ori[C_list, C_list] - C_arr_ori.sum(axis=0)

    return C_arr_ori

In [412]:
def C_energy(shape_q, C_para_dic, trans_matrix_dic):
    C_q = C_para_dic['C_q']
    C_c = C_para_dic['C_c']
    C_qc = C_para_dic['C_qc']
    C_qq = C_para_dic['C_qq']
    row_max = shape_q[0]
    column_max = shape_q[1]
    C_arr_ori = C_matrix_generate(shape_q, C_q, C_c, C_qc)

    n_qi = len(C_qc[0])  # 比特的岛的个数，默认所有比特全同
    n_ci_list = [len(C_qc[i][0]) for i in range(len(C_qc))]  # 每种类型的coupler的岛的个数

    n_q = np.prod(shape_q)
    N_q = n_q * n_qi
    n_c = (2 * row_max * column_max - row_max - column_max)
    N_c = n_c * sum(n_ci_list)
    trans_matrix_add = []

    node_each_list = [n_q] + [n_c] * len(n_ci_list)
    node_list = [sum(node_each_list[:i + 1]) for i in range(len(node_each_list))]

    matrix_list = [trans_matrix_dic['matrix']['q']
                   ] + trans_matrix_dic['matrix']['c']

    for i in range(node_list[-1]):
        node_list_new = node_list + [i]
        node_list_new.sort()
        type_i = node_list_new.index(i)
        trans_matrix_add.append(matrix_list[type_i])
    trans_matrix_all = np.matrix(block_diag(*trans_matrix_add))

    C_arr_ori_trans = np.linalg.inv(
        trans_matrix_all).T * C_arr_ori * np.linalg.inv(trans_matrix_all)

    C_arr_ori_inv = np.linalg.inv(C_arr_ori_trans)

    c_start_index_list = [
        N_q + n_c * sum(n_ci_list[:j]) for j in range(len(n_ci_list))
    ]
    start_index_list = [0] + c_start_index_list

    reserve_index = [trans_matrix_dic['reserve index']['q']
                     ] + trans_matrix_dic['reserve index']['c']
    n_i_list = [n_qi] + n_ci_list

    reserve_index_all = []

    for i in range(1, len(start_index_list)):
        reserve_index_type = [
            list(
                range(start_index_list[i - 1] + reserve_index_i,
                      start_index_list[i], n_i_list[i - 1]))
            for reserve_index_i in reserve_index[i - 1]
            if reserve_index_i is not None
        ]

        reserve_index_all.extend(*reserve_index_type)

    reserve_index_all.sort()

    n_list = range(n_q + n_c)

    C_energy_matrix = C_arr_ori_inv[reserve_index_all, :][:, reserve_index_all]

    C_energy_matrix = constants.e**2 / constants.h * C_energy_matrix * 1e15 / 1e9*1e3 # 单位MHz
    C_energy_matrix[n_list, n_list] = C_energy_matrix[n_list, n_list] / 2
    C_energy_matrix.round(2)
    
    return C_arr_ori, C_energy_matrix

In [363]:
C_para_dic = {
    'C_q': np.array([69.5, 57.85, 46.4, 34.9]),
    'C_c': np.array([117.5, 125]),
    'C_qc': np.array([[[6.33]], [[5.7]]]),
    'C_qq': np.array([[0]])
}

shape_q = [4, 4]

trans_matrix_dic = {
    'matrix': {
        'q': np.array([[1]]),
        'c': [np.array([[1]]), np.array([[1]])]
    },
    'reserve index': {
        'q': [0],
        'c': [[0], None]
    }
}

In [458]:
29.77+5.91

35.68

In [459]:
C_para_dic = {
    'C_q': np.array([70.5, 58.31, 47.05, 35.68]),
    'C_c': np.array([117.9, 125.8]),
    'C_qc': np.array([[[6.099]], [[5.62]]]),
    'C_qq': np.array([[0] ])
}

shape_q = [12, 6]

trans_matrix_dic = {
    'matrix': {
        'q': np.array([[1]]),
        'c': [np.array([[1]]), np.array([[1]])]
    },
    'reserve index': {
        'q': [0],
        'c': [[0], None]
    }
}

In [460]:
C_arr_ori, C_energy_matrix = C_energy(shape_q, C_para_dic, trans_matrix_dic)

In [408]:
##
C_q = C_para_dic['C_q']
C_c = C_para_dic['C_c']
C_qc = C_para_dic['C_qc']
C_qq = C_para_dic['C_qq']
row_max = shape_q[0]
column_max = shape_q[1]
n_qi = len(C_qc[0])  # 比特的岛的个数，默认所有比特全同
n_ci_list = [len(C_qc[i][0]) for i in range(len(C_qc))]  # 每种类型的coupler的岛的个数

N_q = np.prod(shape_q) * n_qi
N_c = (2 * row_max * column_max - row_max - column_max) * sum(n_ci_list)
# 每类coupler的第一个coupler的节点编号
c_start_index_list = [
    N_q +
    (2 * row_max * column_max - row_max - column_max) * sum(n_ci_list[:j])
    for j in range(len(n_ci_list))
]

C_arr_ori = np.matlib.zeros((N_q + N_c, N_q + N_c))
C_list = range(N_q + N_c)
for i in range(row_max * column_max):
    row = i // column_max
    column = i % column_max

    # 当前比特的不同岛的节点编号
    q_i_index_slice = slice(n_qi * i, n_qi * (i + 1))
    q_i_index_list = C_list[q_i_index_slice]

    if row < row_max - 1:
        # 当前比特的下方比特的不同岛的节点编号
        q_j_index_slice = slice(n_qi * (i + column_max),
                                n_qi * (i + 1 + column_max))
        q_j_index_list = C_list[q_j_index_slice]

        C_arr_ori[q_i_index_slice, q_j_index_slice] = -C_qq

        # 当前比特的下方coupler的不同类型不同岛的节点编号
        c_j_index = row * (
            2 * column_max -
            1) + column_max - 1 + column  # 该coupler在本类coupler中的编号

        for j, c_start_index in enumerate(c_start_index_list):

            c_j_index_slice = slice(
                c_start_index + n_ci_list[j] * c_j_index,
                c_start_index + n_ci_list[j] * (c_j_index + 1))
            c_j_index_list = C_list[c_j_index_slice]

            # 当前比特对下方j类coupler
            C_arr_ori[q_i_index_slice, c_j_index_slice] = -C_qc[j]
            # 下方比特对下方j类coupler
            C_arr_ori[q_j_index_slice, c_j_index_slice] = -C_qc[j]
            # 设置下方coupler自电容
            if n_ci_list[j] == 1:
                C_arr_ori[c_j_index_list, c_j_index_list] = C_c[j]  # 设置岛对地电容
            else:
                C_arr_ori[c_j_index_list,
                          c_j_index_list] = C_c[j][:-1]  # 设置岛对地电容
                C_arr_ori[c_j_index_list[0], c_j_index_list[-1]] = -C_c[j][-1]

    if column < column_max - 1:
        # 当前比特的右方比特的不同岛的节点编号
        q_j_index_slice = slice(n_qi * (i + 1), n_qi * (i + 2))
        q_j_index_list = C_list[q_j_index_slice]

        C_arr_ori[q_i_index_slice, q_j_index_slice] = -C_qq
        # 当前比特的右方coupler的不同类型不同岛的节点编号
        c_j_index = row * (2 * column_max -
                           1) + column  # 该coupler在本类coupler中的编号

        for j, c_start_index in enumerate(c_start_index_list):

            c_j_index_slice = slice(
                c_start_index + n_ci_list[j] * c_j_index,
                c_start_index + n_ci_list[j] * (c_j_index + 1))

            c_j_index_list = C_list[c_j_index_slice]
            # 当前比特对右方j类coupler
            C_arr_ori[q_i_index_slice, c_j_index_slice] = -C_qc[j]
            # 右方比特对右方j类coupler
            C_arr_ori[q_j_index_slice, c_j_index_slice] = -C_qc[j]
            # 设置右方coupler自电容
            if n_ci_list[j] == 1:
                C_arr_ori[c_j_index_list, c_j_index_list] = C_c[j]  # 设置岛对地电容
            else:
                C_arr_ori[c_j_index_list,
                          c_j_index_list] = C_c[j][:-1]  # 设置岛对地电容
                C_arr_ori[c_j_index_list[0], c_j_index_list[-1]] = -C_c[j][-1]

    # 设置比特自电容
    nq_around = 0  # 比特周围的比特数
    if row < row_max - 1:
        nq_around += 1
    if row > 0:
        nq_around += 1
    if column < column_max - 1:
        nq_around += 1
    if column > 0:
        nq_around += 1

    if n_qi == 1:
        C_arr_ori[q_i_index_list, q_i_index_list] = C_q[nq_around - 1]
    else:
        C_arr_ori[q_i_index_list, q_i_index_list] = C_q[nq_around - 1][:-1]
        C_arr_ori[q_i_index_list[0],
                  q_i_index_list[-1]] = -C_q[nq_around - 1][-1]

# 设置下三角区域元素
C_down = C_arr_ori.T
C_arr_ori = C_arr_ori + C_down
C_arr_ori[C_list, C_list] = C_arr_ori[C_list, C_list] / 2

# 把非对角元加到对角元上
C_arr_ori[C_list,
          C_list] = 2 * C_arr_ori[C_list, C_list] - C_arr_ori.sum(axis=0)

In [ ]:
##
n_qi = len(C_qc[0])  # 比特的岛的个数，默认所有比特全同
n_ci_list = [len(C_qc[i][0]) for i in range(len(C_qc))]  # 每种类型的coupler的岛的个数

n_q = np.prod(shape_q)
N_q = n_q * n_qi
n_c = (2 * row_max * column_max - row_max - column_max)
N_c = n_c * sum(n_ci_list)
trans_matrix_add = []

node_each_list = [n_q] + [n_c] * len(n_ci_list)
node_list = [sum(node_each_list[:i + 1]) for i in range(len(node_each_list))]

matrix_list = [trans_matrix_dic['matrix']['q']
               ] + trans_matrix_dic['matrix']['c']

for i in range(node_list[-1]):
    node_list_new = node_list + [i]
    node_list_new.sort()
    type_i = node_list_new.index(i)
    trans_matrix_add.append(matrix_list[type_i])
trans_matrix_all = np.matrix(block_diag(*trans_matrix_add))

C_arr_ori_trans = np.linalg.inv(
    trans_matrix_all).T * C_arr_ori * np.linalg.inv(trans_matrix_all)

C_arr_ori_inv = np.linalg.inv(C_arr_ori_trans)

c_start_index_list = [
    N_q + n_c * sum(n_ci_list[:j]) for j in range(len(n_ci_list))
]
start_index_list = [0] + c_start_index_list

reserve_index = [trans_matrix_dic['reserve index']['q']
                 ] + trans_matrix_dic['reserve index']['c']
n_i_list = [n_qi] + n_ci_list

reserve_index_all = []

for i in range(1, len(start_index_list)):
    reserve_index_type = [
        list(
            range(start_index_list[i - 1] + reserve_index_i,
                  start_index_list[i], n_i_list[i - 1]))
        for reserve_index_i in reserve_index[i - 1]
        if reserve_index_i is not None
    ]

    reserve_index_all.extend(*reserve_index_type)

reserve_index_all.sort()

n_list = range(n_q + n_c)

C_energy_matrix = C_arr_ori_inv[reserve_index_all, :][:, reserve_index_all]

C_energy_matrix = constants.e**2 / constants.h * C_energy_matrix * 1e15 / 1e9*1e3 # 单位MHz
C_energy_matrix[n_list, n_list] = C_energy_matrix[n_list, n_list] / 2
C_energy_matrix.round(2)

In [319]:
0.031*1/(5.24/5*7.15/7)**(1/2)

0.02996243132452808

In [318]:
0.0034*1/(5.24/5*5.24/5)**(1/2)

0.003244274809160305

In [468]:
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)
pd.DataFrame(C_energy_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197
0,2.400021e+02,3.073519e+00,1.968089e-02,1.260293e-04,8.070795e-07,5.164783e-09,3.073519e+00,3.944202e-02,3.793631e-04,3.242914e-06,2.598778e-08,1.994774e-10,1.968089e-02,3.793631e-04,4.868379e-06,5.204660e-08,5.007170e-10,4.483361e-12,1.260293e-04,3.242914e-06,5.204660e-08,6.679203e-10,7.498661e-12,7.671984e-14,8.070795e-07,2.598778e-08,5.007171e-10,7.498663e-12,9.623161e-14,1.107382e-15,5.168674e-09,1.999269e-10,4.495817e-12,7.696410e-14,1.111328e-15,1.420596e-17,3.310244e-11,1.495348e-12,3.844392e-14,7.405356e-16,1.188272e-17,1.670399e-19,2.120113e-13,1.095634e-14,3.169939e-16,6.785877e-18,1.197901e-19,1.836500e-21,1.357926e-15,7.902375e-17,2.541217e-18,5.984991e-20,1.152696e-21,1.913902e-23,8.697837e-18,5.629430e-19,1.991943e-20,5.118670e-22,1.068107e-23,1.909295e-25,5.571405e-20,3.970240e-21,1.533022e-22,4.268322e-24,9.592701e-26,1.836657e-27,3.566228e-22,2.770938e-23,1.159117e-24,3.475356e-26,8.367685e-28,1.710396e-29,2.264670e+01,1.450093e-01,9.285493e-04,5.946095e-06,3.807805e-08,2.264670e+01,1.459358e-01,9.404256e-04,6.060287e-06,3.905423e-08,2.514768e-10,1.459358e-01,1.866828e-03,1.793659e-05,1.532463e-07,1.227660e-09,1.450093e-01,1.866828e-03,1.801279e-05,1.544679e-07,1.241782e-09,9.561689e-12,9.404256e-04,1.801279e-05,2.306698e-07,2.463421e-09,2.368382e-11,9.285493e-04,1.793659e-05,2.306698e-07,2.471259e-09,2.382517e-11,2.137768e-13,6.060287e-06,1.544679e-07,2.471259e-09,3.166367e-11,3.551342e-13,5.946095e-06,1.532463e-07,2.463421e-09,3.166367e-11,3.560489e-13,3.648543e-15,3.905423e-08,1.241782e-09,2.382518e-11,3.560490e-13,4.563256e-15,3.807823e-08,1.227681e-09,2.368441e-11,3.551458e-13,4.563441e-15,5.258007e-17,2.516802e-10,9.583346e-12,2.143722e-13,3.660180e-15,5.276508e-17,2.438595e-10,9.442684e-12,2.125664e-13,3.642797e-15,5.265617e-17,6.738069e-19,1.621946e-12,7.190425e-14,1.836969e-15,3.527340e-17,5.648934e-19,1.561783e-12,7.061563e-14,1.817113e-15,3.503446e-17,5.626783e-19,7.916932e-21,1.045273e-14,5.284943e-16,1.517881e-17,3.237380e-19,5.701861e-21,1.000276e-14,5.173382e-16,1.497982e-17,3.209280e-19,5.669804e-21,8.699244e-23,6.736426e-17,3.823769e-18,1.219381e-19,2.859805e-21,5.493569e-23,6.406738e-17,3.731028e-18,1.200662e-19,2.829763e-21,5.453918e-23,9.061887e-25,4.341458e-19,2.732462e-20,9.578199e-22,2.449707e-23,5.096799e-25,4.103669e-19,2.657692e-20,9.410104e-22,2.419644e-23,5.052262e-25,9.036887e-27,2.798002e-21,1.933118e-22,7.386914e-24,2.045962e-25,4.583165e-27,2.628595e-21,1.874240e-22,7.241154e-24,2.017283e-25,4.536290e-27,8.690440e-29,1.801751e-23,1.353357e-24,5.596870e-26,1.668476e-27,4.002958e-29
1,3.073519e+00,2.401827e+02,3.075833e+00,1.969571e-02,1.261242e-04,8.070795e-07,3.941513e-02,3.082483e+00,3.951439e-02,3.799226e-04,3.247113e-06,2.595944e-08,3.790183e-04,1.978178e-02,3.804748e-04,4.879102e-06,5.214234e-08,5.000806e-10,3.239599e-06,1.269596e-04,3.256017e-06,5.219948e-08,6.695154e-10,7.488744e-12,2.595944e-08,8.148941e-07,2.612200e-08,5.025593e-10,7.520716e-12,9.610248e-14,1.996998e-10,5.230851e-09,2.011871e-10,4.515724e-12,7.723331e-14,1.109827e-15,1.493602e-12,3.357987e-11,1.506498e-12,3.864320e-14,7.435413e-16,1.186663e-17,1.094327e-14,2.155862e-13,1.105082e-14,3.188789e-16,6.817255e-18,1.196276e-19,7.892802e-17,1.384198e-15,7.979863e-17,2.558284e-18,6.016073e-20,1.151131e-21,5.622526e-19,8.888133e-18,5.691365e-1

In [464]:
C_energy_matrix[100,100]

149.95427036469633

In [471]:
q_index = 7
c_index = 9
rho = C_energy_matrix[q_index,c_index]/(4*(C_energy_matrix[q_index,q_index]*C_energy_matrix[c_index,c_index])**(1/2))

In [473]:
rho*(5.24/5*5.24/5)**(1/2)*np.sqrt(5*5)

0.00010789340284110163

In [438]:
if column < n_q - 1:
    # 当前比特的右方比特的不同岛的节点编号
    q_j_index_list = slice(n_qi * (i + 1), n_qi * (i + 2))
    C_arr_ori[q_i_index_list, q_j_index_list] = C_qq
    # 当前比特的下方coupler的不同类型不同岛的节点编号
    c_j_index = row * (2 * n_q - 1) + column  # 该coupler在本类coupler中的编号

    for j, c_start_index in enumerate(c_start_index_list):

        c_j_index_list = slice(
            c_start_index + n_ci_list[j] * c_j_index,
            c_start_index + n_ci_list[j] * (c_j_index + 1))
        # 当前比特对下方j类coupler
        C_arr_ori[q_i_index_list, c_j_index_list] = C_qc[j]
        # 下方比特对下方j类coupler
        C_arr_ori[q_j_index_list, c_j_index_list] = C_qc[j]

In [69]:
C_arr_ori[q_i_index_list,:][:,q_i_index_list]

matrix([[0., 0.],
        [0., 0.]])

In [32]:
types_c = len(C_qc)
n_qi = len(C_qc[0])
n_ci = len(C_qc[0][0])

N_q = n_q**2 * n_qi
N_c = 2 * n_q * (n_q - 1) * n_ci * types_c

C_arr_ori = np.matlib.zeros((N_q + N_c, N_q + N_c))

    

In [44]:
i= 5
row = i % n_q
column = i // n_q

In [46]:
row*(2*n_q-1)+n_q-1+column

11

In [42]:
C_arr_ori[n_qi * i:n_qi * (i + 1), n_qi * i - n_q:n_qi * (i + 1) - n_q] = C_qq
C_arr_ori[range(5,6), n_qi * i - n_q:n_qi * (i + 1) - n_q]

matrix([[5.]])

In [ ]:
np.set_printoptions(threshold=np.inf)
C_arr_ori

In [467]:
10.833 - 1.666


9.167